In [5]:
import pandas as pd
import hiplot as hip   
from scipy.stats import ttest_ind
import os 

os.chdir('src')
from toolkit.etl_toolkit import ingest_data
from toolkit.analysis_toolkit import inspect_distribution, calculate_drivetime

In [9]:
#2.8 GB of RAM consumed -- it could be impossible to load the full dataset at once. Using dask for modelling should be considered
static_draft = ingest_data('2018', '01').compute()
#Creating the graph for sister dataset. It will be used for future comparisons
sister_draft = ingest_data('2019', '01')
#From now on the data (resp. column) drawn from a next year I will refer to as ,,sister data" (resp. ,,sister column") 

KeyboardInterrupt: 

In [ ]:
len(set(static_draft.index))

In [ ]:
static_draft.shape

There is less than ten times less ,,unique" records (based on index). That is odd, it is impossible for the index to represent the specific driver (there would be less than twenty drivers in NYC city then). We will inspect that matter.

In [ ]:
static_draft.loc[1, :]

There is no clear similarity in these rows, except an improvement surcharge which is said to take only values from {0, 0.5}, so cannot be related to an index. Nonethless from now on I will be reffering to records with the same index as unique trips.

### 1. Pre-inspecting the data based on January rides.
### 1.1 Domain understanding: Inspecting the variables

In [ ]:
static_draft.head()

In [ ]:
static_draft.isnull().sum()

In [ ]:
#Reference for choosing wether the data is stored in the suitable format. 
#With EDA results it will be used for a proper pipeline design.
static_draft.dtypes

### 1.1.1 Column insights: VendorID
Indicates the **TPEP** (Taxicab Passenger Enhancement Program) provider of a record. 
Unique values:
    
   1) 1 := Creative Mobile Technologies, LLC; 
    
   2) 2 := VeriFone Inc.
    
Treated as categorical variable. I will now proceed with inspecting its relevancy i.e. the impact that it makes on distribution of the rest of the data.

In [ ]:
static_draft['VendorID'].value_counts()/len(static_draft)

In [ ]:
#For locations
hip.Experiment.from_dataframe(static_draft.iloc[:100, :].loc[:, ['VendorID', 'PULocationID', 'DOLocationID']]).display()

It seems that there are no separate districts in which the both providers operate. 

### 1.1.2 Column insights: tpep_pickup_datetime, tpep_dropoff_datetime
The date and time when the meter was engaged/disengaged.

Format of the string: 'yyyy-mm-dd hh:mm:ss'
Will be used to create an indicator wether it was a night ride as well as for calculating how long it lasted.

In [ ]:
static_draft.loc[:, 'drivetime'] = calculate_drivetime(static_draft)

In [ ]:
inspect_distribution(static_draft.head(500000)['drivetime'], True, calculate_drivetime(sister_draft.head(500000)))

Data seems to be concentrated on a shorter rides (time given in minutes), which is of high importance taking under consideration initial fee's, more quick drives means shorter distances, less time spent on returning to the city centre and more engaging fees collected. Nonetheless the outliers should be inspected. But first the equal expected values hypothesis will be tested.

In [ ]:
#T test results.
ttest_ind(static_draft.head(500000)['drivetime'], calculate_drivetime(sister_draft.head(500000)))

It will be assumed that the year-to-year expected drivetime is changing - decreasing.

In [ ]:
static_draft.loc[static_draft['drivetime'] > 100, :].sort_values('fare_amount')

It seems that the outliers may be related both to the personal errands or even trips (negative fare) and longer drives. The odd thing is that some of the heavily charged services are not related to driving the passengers at all. That is why I would recomend not using these records in EDA and modelling processes as they need more care.  

In [ ]:
inspect_distribution(static_draft.loc[static_draft['drivetime'] < 100, :].head(1000)['drivetime'], True, [x if x < 100 else None for x in calculate_drivetime(sister_draft.head(1000))])

In [ ]:
#Managing the outlying trips, based on the drivetime
static_draft = static_draft.loc[static_draft['drivetime'] < 100, :]

In [ ]:
#6.6. GB consumed
sister_draft = sister_draft.compute()
sister_draft = sister_draft.loc[calculate_drivetime(sister_draft) < 100, :]

### 1.1.3 Column insights: passenger_count
The number of passengers, driver included. Should be checked for any anomalies, maybe the relation to the length of the ride and drivespeed should be inspected.

In [ ]:
inspect_distribution(static_draft['passenger_count'][1000:20000], True, sister_draft['passenger_count'][1000:20000])

There seems to be some personal or anomaly trips included in the data. Let's inspect them before dropping.

In [ ]:
static_draft.loc[static_draft['passenger_count'] == 0, :]

In [ ]:
static_draft.loc[static_draft['passenger_count'] == 0, ['VendorID', 'payment_type']].value_counts()

In [ ]:
pd.crosstab(static_draft['VendorID'], static_draft['passenger_count'])

Occurences of the 'zero-passengers' records are related to the VendorID, maybe there were some difficulties in collecting this data? These records will be excluded from modelling but included in the further EDA for the rest of related variables seems intact. 

### 1.1.4 Column insights: store_and_fwd_flag

This flag indicates whether the trip record was held in vehicle memory before sending to the vendor, because the vehicle did not have a connection to the server. 
Unique values:
1. Y= store and forward trip

2. N= not a store and forward trip

This column will not be used in further analysis for its highly probable irrelevance. 

In [ ]:
static_draft = static_draft.drop('store_and_fwd_flag', 1)
sister_draft = sister_draft.drop('store_and_fwd_flag', 1)

### 1.1.5 Column insights: payment_type
A numeric code signifying how the passenger paid for the trip.
Unique values: 

1. 1= Credit card

2. 2= Cash

3. 3= No charge

4. 4= Dispute

5. 5= Unknown

6. 6= Voided trip

If I were to express my opinion I would call this column the one of the most important one. That is because that grim ''dispute" value. Intuitively it could be related to default in payment, in any case this matter demands an inquiry.

In [ ]:
inspect_distribution(static_draft['payment_type'][2000:30000], True, sister_draft['payment_type'][2000:30000])

In [ ]:
static_draft['payment_type'].value_counts()

In [ ]:
inspect_distribution(static_draft.loc[static_draft['payment_type'] == 4, 'fare_amount'])

It looks like the 'dispute' value not necessarily indicates that the whole charge was lost, but we encounter some negative values. Now the average fare and assumed loss on these trips will be calculated. 

In [ ]:
static_draft.loc[static_draft['fare_amount'] < 0, ['fare_amount', 'payment_type']].groupby('payment_type').mean()

### 1.1.6 Column insights: ratecodeID

RatecodeID
The final rate code in effect at the end of the trip.

Unique values:

1= Standard rate

2=JFK

3=Newark

4=Nassau or Westchester

5=Negotiated fare

6=Group ride

It is worth checking if predicting the negotiated fare would have any business value.

In [ ]:
static_draft['RatecodeID'].value_counts()/len(static_draft)

There are only a few negotiated fares and category '99' which origins are rather blurry, the column will be dropped before EDA.

### 1.1.7 Column insights: financial data

1. fare_amount
The time-and-distance fare calculated by the meter.

2. extra
Miscellaneous extras and surcharges. Currently, this only includes the 0.50 dollar and 1 dollar rush hour and overnight charges.

3. mta_tax
0.50 dollar MTA tax that is automatically triggered based on the metered rate in use.

4. improvement_surcharge
0.30 dollar improvement surcharge assessed trips at the flag drop.

5. tip_amount
Tip amount – This field is automatically populated for credit card tips. Cash tips are not included.

6. tolls_amount
Total amount of all tolls paid in trip.

7. total_amount
The total amount charged to passengers. Does not include cash tips.

I will use this data to check wether the "dispute" payment type really has any impact on the final charge. Also the presence of the tip amount-related information could indicate that there was a card payment, which could be used to fill some of the missing payment type data. 

In the next notebooks I am planning to use just "fare_amount", "tip_amount" and "extra" variables for their relevance.

In [ ]:
len(static_draft.loc[static_draft['fare_amount'] < 0, :])/len(static_draft)

Negative charge records are sparse and for their sparsity the will not be included in further analysis. Their relation to pickup and dropoff location also seems unclear.

### 1.1.8 Column insights: PULocationID, DOLocationID

Respectively: TLC Taxi Zone in which the taximeter was engaged, TLC Taxi Zone in which the taximeter was disengaged.

These will be used for modellig trip duration/speed/total charge. There is lookup table provided by TLC, it could be used to decypher the Borough, Zone and the information which type of taxi services the specific zone.

In [ ]:
zone_lookup = pd.read_csv('https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv')

In [ ]:
zone_lookup.nunique()

In [ ]:
hip.Experiment.from_dataframe(static_draft.iloc[:10000, :].loc[:, ['total_amount', 'PULocationID', 'DOLocationID']]).display()

Obviously there is correlation between pickup/dropoff location and the total amount charged but could it be simplified by using the Boroughs?

In [ ]:
zone_dictionary = pd.Series(zone_lookup.Borough.values,index=zone_lookup.LocationID).to_dict()
sample_draft = static_draft.head(10000).loc[:, ['total_amount', 'PULocationID', 'DOLocationID']]

sample_draft = sample_draft.replace({"PULocationID":zone_dictionary}).replace({"DOLocationID":zone_dictionary})

hip.Experiment.from_dataframe(sample_draft.iloc[:100, :].loc[:, ['total_amount', 'PULocationID', 'DOLocationID']]).display()

It can be easily spotted that both least and most expensive trips routes lies entirely in Manhattan. It is striking that some of the Boroughs are just disproportionately large. For a more in depth analysis it would be an asset to collect the data regarding boroughs size and population. In the next notebook I am going to engineer the feature "borough" size by simply counting the IDs for every Borough in the zone lookup table.

The pickups from zone other than yellow taxi zone will not be taken into account as formally drivers cannot take passengers from other taxi zone. Because of that I expect some fare anomalies in these cases.

### Pre- EDA Summary 

#### Matters that needs to be tended in the preprocessing:

1. Dropping unused columns

2. Cleaning missing values if any encountered 

3. Converting timestamp columns to the proper type and format 

4. Cleaning negative trip durations, trip distances, 

5. Cleaning negative fares

6. Cleaning the trips with no passengers

7. Encoding the variables that should be categorical

#### Feature engineering:

1. Calculate Borough sizes using number of zones that every one of them contains

2. Calculate average drive speed

3. Calculate the drivetime.

4. Create the indicator wether it was a night/rush hour course.

5. Create the indicator showing if the trip happened during the weekend

6. Create a season indicator (for models trained on many months).

7. Optionally merge the outlying payment types together into the "uncommon" category.

#### Features to keep from the original dataset:

1. PULocationID and DOLocationID
2. tpep_pickup_datetime and tpep_dropoff_datetime
3. passenger_count
4. trip_distance
5. payment_type
6. fare_amount
7. extra
8. tip_amount

### Handling anomalies

Erase rows where: 

1. PULocationID or DOLocationID is in {0, 264, 265}

2. Total amount is negative

3. "extra" value is negative 

4. "tip_amount" is negative

5. Trip lasts longer than 100 minutes or its duration is less than 0 minutes.

6. Erase rows with missing values.

Useful links. 
Dictionary: https://www1.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf

Lookup table regarding IDs: https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv